In [ ]:
from tkinter import *
from tkinter import filedialog

# A function to allow the user to select the folder contianing the subfolders of images.
# Function input arg 1: create_all_videos [bool] --> When 0, asks for the Well diirectory. When 1, asks for the directory containing the parents.
# Function input arg 2: test [bool] --> When 1, will change the gui title to that of the test gui.
# Function output 1: The path of the folder selected by the user. 
def folder_selection_dialog(create_all_videos = 0,
                            test = 0):
    root = Tk()
    if test:
        root.title('Please select the "test" folder within the downloaded package.')
        root.filename = filedialog.askdirectory(initialdir="/", title="Please select the 'test' folder within the downloaded package.")
    else:
        if create_all_videos == 0:
            root.title('Please select the well directory for which you want to make a video')
            root.filename = filedialog.askdirectory(initialdir="/", title="Please select the directory containing the subfolders of image data")
        elif create_all_videos == 1:
            root.title('Please select the folder containing the parent directories')
            root.filename = filedialog.askdirectory(initialdir="/", title="Please select the directory containing the subfolders of image data")
        else:
            raise Exception("Value of create_all_videos must be 0 or 1, and cannot be int type.")
    selected_directory = root.filename
    root.destroy()

    return selected_directory

In [ ]:
import os as os

# A function to create a list of image paths, such that these images can later be used to make a movie. 
# Function input arg 1: selected_directory [string] --> The directory containing the subfolders of images. 
# Function input arg 2: file_type [string] --> The file extension that you want to detect. 
# Function output 1: image_paths [list] --> A list of the image paths, where each path is a string. 
def list_image_paths(selected_directory,
                     file_type = '.jpg'):
    
    # First, list the subfolders in the well directory.
    subfolders = [_ for _ in os.listdir(selected_directory)]

    # Iterate through the list of subfolders and extract the images contained within them. 
    image_paths = []
    for i in range(len(subfolders)):
        
        current_subfolder = subfolders[i]
        subfolder_dir = os.path.join(selected_directory, current_subfolder, 'DCIM')
        images_in_subfolder = [os.path.join(subfolder_dir, image) for image in os.listdir(subfolder_dir) if image.endswith(file_type)]
        image_paths.append(images_in_subfolder)
        
    image_paths = [image for sublist in image_paths for image in sublist]
    
    return image_paths

In [ ]:
import os as os

# A function to return the list of well folders, such that images with can later be used to make a movie. 
# Function input arg 1: selected_directory [string] --> The directory containing the well folders of images. 
# Function output 1: well_paths [list] --> A list of the well_folder paths, where each path is a string. 
def list_well_paths(selected_directory):
    
    # First, list the subfolders in the well directory.
    subfolders = [_ for _ in os.listdir(selected_directory)]

    # Iterate through the list of village_directories and extract the well_directories contained within them. 
    well_paths = []
    for i in range(len(subfolders)):
        
        subfolder_dir = os.path.join(selected_directory, subfolders[i])
        well_directories = [os.path.join(subfolder_dir, _) for _ in os.listdir(subfolder_dir)]
        well_paths.append(well_directories)
        
    well_paths = [path for sublist in well_paths for path in sublist]
    
    return well_paths

In [ ]:
import os 
import cv2
import time as time 

# A function to take the list of image paths, load in said images, and convert them to a movie. 
# Function input arg 1: selected_directory [string] --> The well directory, as previously selected. 
# Function input arg 1: create_all_videos [bool] --> When 0, asks for the Well diirectory. When 1, asks for the directory containing the parents.
# Function input arg 3: frame_rate [int] --> The desired frame rate. 
# Function input arg 4: movie_name [string] --> Your desired file name, including the file extension. Tested for .avi and .mp4. 
# Function output 1: The movie will be saved to 'selected_directory'. 
def create_movie(selected_directory,
                 create_all_videos = 0,
                 file_type = '.jpg',
                 frame_rate = 1,
                 movie_name = 'my_movie_name.avi'):
    
    if create_all_videos == 0:
    
        image_paths = list_image_paths(selected_directory,
                                      file_type = file_type)
    
        # Change the current working directory to the well folder. 
        os.chdir(selected_directory)

        # Create the video. 
        frame = cv2.imread(image_paths[0])
        height, width, layers = frame.shape
        video = cv2.VideoWriter(movie_name, 0, frame_rate, (width,height))
        
        # Start recording the processing duration.
        t = time.time() 
        
        for i in range(len(image_paths)):
            t = time.time() # Start recording the processing duration.

            # Print the progress as images are 'stiched' together over time. 
            if i%100 == 0:
                percentage = (i / len(image_paths))*100
                print(f"Progress = {percentage}%")
            video.write(cv2.imread(image_paths[i]))

        cv2.destroyAllWindows()
        video.release()
        
        elapsed = time.time() - t # Calculate the processing duration.
        print(f"------------\nYour video, '{movie_name}' (FPS = {frame_rate}), has been saved to:\n\n{selected_directory}\nTime taken = {elapsed/60} min\n------------\n\n")
        
    if  create_all_videos == 1: 
        
        well_paths = list_well_paths(selected_directory)
        
        for i in range(len(well_paths)):
            
            image_paths = list_image_paths(well_paths[i],
                                          file_type = file_type)

            # Change the current working directory to the well folder. 
            os.chdir(selected_directory)

            # Create the video. 
            frame = cv2.imread(image_paths[0])
            height, width, layers = frame.shape
            video = cv2.VideoWriter(movie_name, 0, frame_rate, (width,height))

            # Start recording the processing duration.
            t = time.time() 

            for i in range(len(image_paths)):
                t = time.time() # Start recording the processing duration.

                # Print the progress as images are 'stiched' together over time. 
                if i%100 == 0:
                    percentage = (i / len(image_paths))*100
                    print(f"Progress = {percentage}%")
                video.write(cv2.imread(image_paths[i]))

            cv2.destroyAllWindows()
            video.release()

            elapsed = time.time() - t # Calculate the processing duration.
            print(f"------------\nYour video, '{movie_name}' (FPS = {frame_rate}), has been saved to:\n\n{selected_directory}\nTime taken = {elapsed/60} min\n------------\n\n")


In [ ]:
# Things to do. 
# 1 - stop functions from thinging previously exported videos are folders. 
# 2 - auto naming of videos. With village, well, FPS, and datetime. 
# 3 - make videos save to well folder. 